# BPIC2017 numeric tryout (unsuccessful, redundant)


In [1]:
from CI_Experiments.config import PROJECT_DIR
import pandas as pd
import matplotlib.pyplot as plt
from ylearn import Why

2023-04-20 13:10:04,889 - /Users/lukas/opt/anaconda3/envs/ci_experiments/lib/python3.8/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [2]:
train_path = PROJECT_DIR / "data/other_data/BPIC2017_1/activities_numeric2/train.csv"
test_path = PROJECT_DIR / "data/other_data/BPIC2017_1/activities_numeric2/test.csv"

In [3]:
train_path2 = PROJECT_DIR / "data/other_data/BPIC2017_1/activities2/train.csv"
test_path2 = PROJECT_DIR / "data/other_data/BPIC2017_1/activities2/test.csv"

In [ ]:
train_data = pd.read_csv(train_path)
train_data.head()

In [ ]:
test_data = pd.read_csv(test_path)
test_data.head()

In [ ]:
why3 = Why(discrete_outcome=True, discrete_treatment=False, identifier='pgm', random_state=23, estimator='bound')
r3 = why3.identify(train_data, 'Outcome')
r3

In [ ]:
why3 = Why(discrete_outcome=True, discrete_treatment=False, identifier='pgm', random_state=23, estimator='div')
why3.fit(train_data, 'Outcome', treatment='CreditScore')

In [ ]:
cov = ['RequestedAmount',
  'FirstWithdrawalAmount',
  'MonthlyCost',
  'NumberOfTerms',
  'OfferedAmount',
  'open_cases',
  'Activity_A_Accepted',
  'Activity_A_Cancelled',
  'Activity_A_Complete',
  'Activity_A_Concept',
  'Activity_A_Create Application',
  'Activity_A_Denied',
  'Activity_A_Incomplete',
  'Activity_A_Pending',
  'Activity_A_Submitted',
  'Activity_A_Validating',
  'Activity_O_Accepted',
  'Activity_O_Cancelled',
  'Activity_O_Create Offer',
  'Activity_O_Created',
  'Activity_O_Refused',
  'Activity_O_Returned',
  'Activity_O_Sent (mail and online)',
  'Activity_O_Sent (online only)',
  'Activity_W_Assess potential fraud',
  'Activity_W_Call after offers',
  'Activity_W_Call incomplete files',
  'Activity_W_Complete application',
  'Activity_W_Handle leads',
  'Activity_W_Personal Loan collection',
  'Activity_W_Shortened completion ',
  'Activity_W_Validate application']

In [ ]:
from xgboost.sklearn import XGBRegressor

In [ ]:
from ylearn.estimator_model.double_ml import DoubleML
x = XGBRegressor()
y = XGBRegressor()
dml = DoubleML(x_model=x, y_model=y, random_state=23, is_discrete_treatment=False)
dml.fit(train_data, 'Outcome', treatment='CreditScore', covariate=cov)

In [ ]:
dml.estimate(data=test_data, quantity='CATE')

In [ ]:
dml.estimate(data=test_data, quantity='ATE')

In [ ]:
def calc_and_plot_qini(why_, test_data_, treatment):
    uplift = why_.uplift_model(test_data_, treatment=treatment)
    qini_ = uplift.get_qini()
    top_ = qini_[qini.columns[0]].iat[-1]
    idx_ = qini_.index[-1]
    plt.plot(qini_.index, qini_[qini_.columns[0]], label=treatment)
    plt.plot((0, idx_), (0, top_), label='random')
    plt.xlabel('Population')
    plt.ylabel('Qini')
    plt.legend()
    plt.show()

In [ ]:
calc_and_plot_qini(why3, test_data, 'CreditScore')

In [ ]:
why = Why(discrete_outcome=True, discrete_treatment=False, identifier='notears', random_state=23, estimator='bound')
r = why.identify(train_data, 'Outcome')
r

In [ ]:
why2 = Why(discrete_outcome=True, discrete_treatment=False, identifier='gcastle', random_state=23, estimator='bound')
r2 = why.identify(train_data, 'Outcome')
r2

## Only activites (target encoded)

In [ ]:
train_data2 = pd.read_csv(train_path2)
train_data2.head()

In [ ]:
why21 = Why(discrete_outcome=True, discrete_treatment=False, identifier='pgm', random_state=23, estimator='bound', 
           identifier_options={'max_cond_vars': 50})
r21 = why21.identify(train_data2, 'Outcome')
r21

In [ ]:
why22 = Why(discrete_outcome=True, discrete_treatment=False, identifier='notears', random_state=23, estimator='bound')
r22 = why22.identify(train_data2, 'Outcome')
r22

In [ ]:
why23 = Why(discrete_outcome=True, discrete_treatment=False, identifier='gcastle', random_state=23, estimator='bound')
r23 = why23.identify(train_data2, 'Outcome')
r23

## Uplift

In [ ]:
# import approaches
from sklift.models import SoloModel
from lightgbm import LGBMClassifier
import numpy as np

In [ ]:
data_ = train_data2.copy()
data_.head()

In [ ]:
test_data2 = pd.read_csv(test_path2)
test_data2.head()

In [ ]:
data_['treatment'] = np.where(train_data['Activity_W_Validate application'] > 0, 1, 0)
data_['treatment'].head()

In [ ]:
X_cols = list(data_.columns)
X_cols.remove('Activity_W_Validate application')
X_cols

In [ ]:
data_ = data_[X_cols]
data_.head()

In [ ]:
cov = [
  'Activity_A_Accepted',
  'Activity_A_Cancelled',
  'Activity_A_Complete',
  'Activity_A_Concept',
  'Activity_A_Denied',
  'Activity_A_Incomplete',
  'Activity_A_Pending',
  'Activity_A_Submitted',
  'Activity_A_Validating',
  'Activity_O_Cancelled',
  'Activity_O_Create Offer',
  'Activity_O_Created',
  'Activity_O_Refused',
  'Activity_O_Returned',
  'Activity_O_Sent (mail and online)',
  'Activity_O_Sent (online only)',
  'Activity_W_Assess potential fraud',
  'Activity_W_Call after offers',
  'Activity_W_Call incomplete files',
  'Activity_W_Handle leads',
  'Activity_W_Personal Loan collection',
  'Activity_W_Shortened completion '
]

In [ ]:
why_31 = Why(random_state=23)
why_31 = why_31.fit(data_.copy(), outcome='Outcome', treatment='treatment', covariate=cov)

In [ ]:
t_data_ = test_data2.copy()
t_data_.head()

In [ ]:
t_data_['treatment'] = np.where(t_data_['Activity_W_Validate application'] > 0, 1, 0)
t_data_['treatment'].head()

In [ ]:
t_data_ = t_data_[X_cols]
t_data_.head()

In [ ]:
um = why_31.uplift_model(test_data=t_data_, treatment='treatment', random='random', name='Activity_W_Validate application=1')

In [ ]:
um.get_qini()

In [ ]:
qini = um.get_qini()
top = qini[qini.columns[0]].iat[-1]
idx = qini.index[-1]
plt.plot(qini.index, qini[qini.columns[0]], label='1')
plt.plot((0, idx), (0, top), label='random')
plt.xlabel('Population')
plt.ylabel('Qini')
plt.legend()
plt.show()

In [ ]:
um.plot_qini()

In [ ]:
t_data = test_data2.copy()
t_data['treatment'] = np.where(t_data['Activity_W_Validate application'] > 0, 1, 0)
t_data['treatment'].head()

In [ ]:
t_data_X = t_data[X_cols]
t_data_X.head()

In [ ]:
t_Y = t_data['Outcome']
t_Y.head()

In [ ]:
t_treat = t_data['treatment']
t_treat.head()

In [ ]:
# define models
estimator = LGBMClassifier(n_estimators=10)

# define metamodel
slearner = SoloModel(estimator=estimator)


# fit model
slearner.fit(
    X=t_data_X.to_numpy(),
    y=t_Y.to_numpy(),
    treatment=t_treat.to_numpy(),
)

# predict uplift
uplift_slearner = slearner.predict(t_data_X.to_numpy())


In [ ]:
from sklift.viz import plot_qini_curve
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_title('Qini curves')

plot_qini_curve(
    t_Y, uplift_slearner, t_treat, name='Slearner', ax=ax
);

In [ ]:
t_data_X